<a href="https://colab.research.google.com/github/yoshithedinosaur/confluency-analysis-tool/blob/main/confluency_tool_v0_82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Confluency Tool v0.82**
**Current features:**
  - Batch analysis of image sets in directory
  - Progress indicator
  - Plotting of confluency vs time for the selected image set
  - 11/8/22 - Added "Enable detailed outputs" feature to improve data analysis and troubleshooting
  - 11/8/22 - Added y-error bars to plots
  - 11/9/22 - Improved image set  selection and added ability to merge analyses between sets

**Planned additions/improvements (ordered by priority):**
  - Write all data to and read from google sheets
  - Ability to run all image sets in directory
    - And only run sets that are not already in a designated data sheet unless specified
  - Automatically calculate doubling time
  - ~~Some way to inspect which, if any, images are giving inaccurate confluency measurements~~
  - An "Abort" button that doesn't require restarting the runtime
  - Ability to upload and analyze single images
  - Ability to upload and analyze folders of images from local device
  - Ability to select which directory to use
  - Time estimates for how long the analysis will take
  - ~~A "verbose" option to show all images before and after processing~~
  - ~~A nifty little progress bar~~

**Future versions:**
  - Modularity for image pre-processing and post-processing steps
  - General optimizations
  - Training or selecting the machine-learning model from within the tool

**Known issues:**
  - The trained ML model is highly inaccurate with certain image sets (specifically with some of the "Growth Curve" images)
  - Dead cells are not accounted for

All in all, about *82% done* with this code.

### Preamble

In [ ]:
#PREAMBLE
import cv2
from datetime import date, datetime, time, timedelta
from functools import partial
import glob
from google.colab import files
from google.colab import auth
auth.authenticate_user() # authenticate user
import io
import IPython
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import joblib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from PIL import Image
from scipy.fft import fftn, fftshift
from skimage import data, segmentation, feature, future, color, morphology
from skimage.color import rgb2gray
from skimage.filters import difference_of_gaussians, window
from sklearn import datasets, svm
from sklearn.ensemble import RandomForestClassifier
from typing import Dict, Tuple, List, Union, Optional
from tqdm.auto import tqdm

# Tool to read files from google drive
from google.colab import drive
drive.mount('/content/drive')

# Checks that path is found in google drive
!ls "/content/drive/Shareddrives/IPRO Fall 2022 (iGEM)/SCRC - Cholesterol Project"

# DECLARE IMAGE DIRECTORY **IMPORTANT**
image_dir = "/content/drive/Shareddrives/IPRO Fall 2022 (iGEM)/SCRC - Cholesterol Project/Cell Team/Cell Pictures/"

Mounted at /content/drive
'Animation Storyline.gdoc'	     'Cholest.IPRO.Supply log.gsheet'
'Assay Team'			     'Goals .pptx'
'Before we start'		     'IPRO Poster.gslides'
'Cell Team'			     'Regulatory Response Team'
'Cholesterol Biosensor.gslides'      'Team Charter.gdoc'
'Cholesterol Biosensor Poster.pptx'  'Team Members.xlsx'


### FUNCTIONS

In [ ]:
#FUNCTIONS
clf = joblib.load(image_dir + "Confluency Tool Python Scripts/model.joblib") # pick model


# Use "clf" model from ML algorithm
def region_finding(image: Union[Image.Image, np.ndarray]) -> Union[Image.Image, np.ndarray]:
  """This function finds the regions of cells based on the input model ("clf") for the input image

  Parameters
  ----------
  image : Image.Image, np.ndarray
    The band pass filtered image

  Returns
  -------
  processed_image : Image.Image
    The output image processed by the ML model
  """

  sigma_min = 1
  sigma_max = 16
  features_func = partial(feature.multiscale_basic_features, intensity=True, edges=False, texture=True, sigma_min=sigma_min, sigma_max=sigma_max)
  features = features_func(image)
  processed_image = future.predict_segmenter(features, clf)
  return processed_image




def fourier_bandpass(image: Image.Image) -> np.ndarray:
  """"This function performs a band pass filter to enhance edges of an image

  Parameters
  ----------
  image : Image.Image
    The original image

  Returns
  -------
  filtered_image : np.ndarray
    The edge-enhanced result image
  """

  image = asarray(image)
  wimage = image * window('hann', image.shape)  # window image to improve FFT
  filtered_image = difference_of_gaussians(image, 1.5)
  filtered_wimage = filtered_image * window('hann', image.shape)
  return asarray(filtered_image)




def denoise(image: np.ndarray,object_size=5000,hole_size=5000) -> np.ndarray:
  """This function denoises the imput image

  Parameters
  ----------
  image : np.ndarray
    Image to denoise

  Returns
  -------
  result_im : np.ndarray
    The denoised result image
  """

  result_im = morphology.remove_small_objects(image==2,object_size)
  result_im = morphology.remove_small_holes(result_im,hole_size)
  return result_im




def label_im_dirs(image_dir: str) -> Dict[str, List[str]]:
  """This function finds all images and labels each to the corresponding folder

  Parameters
  ----------
  image_dir : str
    Name of directory for all images that analysis is desired for

  Returns
  -------
  data_dict : Dict[str, List[str]]
    Dictionary of all image file paths grouped by their descriptive names/image set
  """

  im_paths = glob.glob(image_dir + "**/*.jpg", recursive=2)
  data_dict = {}
  old_dir_desc = ''

  for im_name in im_paths:
    im_desc = im_name.split('/')  # puts image path into list of strings using '/' as the delimeter

    if (len(im_desc[len(im_desc) - 2]) == 8) and not (im_desc[len(im_desc) - 1]).isnumeric():  # 8 is the length for YYYYMMDD directory format
      desc_append_index = len(im_desc) - 2   # penultimate string is the YYYYMMDD format directory name which is superfluous and non-descriptive of the image set
    else:
      desc_append_index = len(im_desc) - 4   # incase files were not saved in the YYYYMMDD format but in YYYY/MM/DD format instead, those three strings are omitted
    
    dir_desc = ' '.join(im_desc[8:desc_append_index])

    if old_dir_desc == dir_desc:
      data_dict[dir_desc].append(im_name)
    else:
      data_dict.update({dir_desc : [im_name]})
    
    old_dir_desc = dir_desc
  return data_dict




def confluency(image: np.ndarray) -> float:
  """A function that measures the confluency from a pre-processed segmented image

  Parameters
  ----------
  image : np.ndarray
    The segmented image that has been sent through the region finder

  Returns
  -------
  cfy : float
    A float describing the confluency of the input image, relative to unity
  """

  unique, counts = np.unique(image, return_counts=True)
  if len(unique) == 2:
    final_counts = dict(zip(unique, counts))
    cfy = final_counts[1]/(final_counts[0]+final_counts[1])
  else:
    cfy = float(unique)
  return cfy




def datetime_metadata(image: Image.Image) -> datetime:
  """A function to retrieve the date-time metadata of the input image

  Parameters
  ----------
  image : Image.Image
    The image to read metadata from

  Returns
  -------
  fdate : datetime
    The date-time metadata in datetime format
  """

  exif = image.getexif()
  creation_time = exif.get(36867)

  # Splitting date time data to reformat for use
  ydate, dtime = creation_time.split(" ", 1)
  year, month, day = ydate.split(":", 2)
  hour, minute, second = dtime.split(":", 2)
  fdate = datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
  return fdate




def relative_days(data_list: List[Tuple[float, datetime]]) -> List[Tuple[float, timedelta]]:
  """This function converts the data's dates to days elapsed since the chronologically first coordinate

  Parameters
  ----------
  data_list : List[Tuple[float, datetime]]
    -data_list = list of tuples containing the coordinates (x, t), where t is the absolute date of the data point x

  Returns
  -------
  data_t_trunc : List[Tuple[float, timedelta]]
    Same list of tuples but t is now truncated to relative time from the minumum time (date) in the tuple in the list
  """
  confluency_coord = [coord[0] for coord in data_list]
  date_time_coord = [coord[1] for coord in data_list]
  min_date = min(date_time_coord)

  data_t_trunc = []
  
  for creation_time in date_time_coord:
    current_index = date_time_coord.index(creation_time)
    tdelta = creation_time - min_date
    data_t_trunc.append((confluency_coord[current_index], tdelta.days + tdelta.seconds/86400))

  return data_t_trunc



def linear_prop_err():
  pass

### Body

#### Analysis

In [ ]:
#GUI

# use glob, get file path. set key for image to descriptive name for each image
# parse available folders
# gui to select the folders containing images
# fourier_bandpass -> region_finding -> denoise
# plot for selected folder
# save the plot/data
# dictionary of lists of tuples of confluency and time

#print(label_im_dirs(image_dir).keys())
#test_list = [(0.1,datetime(2022, 10, 28, 4, 6, 5)),(0.2,datetime(2022, 10, 29, 8, 59, 10)),(0.5,datetime(2022, 11, 1, 22, 12, 6))]
#print(relative_days(test_list))
#test_image = label_im_dirs(image_dir)['Thaw'][15]
#im = Image.open(test_image)
#fb_im = fourier_bandpass(im)
#rf_im = region_finding(fb_im)
#dn_im = denoise(rf_im)
#plt.imshow(dn_im, cmap='gray')

im_dict = label_im_dirs(image_dir)
im_keys = list(im_dict.keys())

data_dict = {im_set : [] for im_set in im_keys}
image_sets = sorted(list(im_dict.keys()))
#image_sets[0] = "Uploaded Image(s)"
verbose_ims = []

def im_set_analyze(image_set):

  display("Processing images...")
  if len(image_set) > 1:
    data_dict['+'.join(list(image_set))] = []
  #for image_paths in tqdm(sorted(im_dict[image_set])):
  #for key in tqdm(list(image_set), desc='Progress'):
  for key in list(image_set):
    for image_paths in tqdm(im_dict[key], desc=key):
        im = Image.open(image_paths)
        fb_im = fourier_bandpass(im)
        rf_im = region_finding(fb_im)
        dn_im = denoise(rf_im)
        cfy = confluency(dn_im)
        dt = datetime_metadata(im)
        data_dict['+'.join(list(image_set))].append( (cfy, dt) )

        # If verbose mode checkbox is checked, shows every image processed
        if verbose_mode.value:
          #encoded_im = np.array(cv2.imencode('.jpg', 255*dn_im.astype(np.float64))[1])
          #byte_im = encoded_im.tobytes()
          #verbose_ims.append(widgets.Image(
          #  value=IPython.display.Image(byte_im).data,
          #  format='jpg'
          #))
          plt.imshow(im, 'gray', interpolation='none')
          if cfy == 1.0:
            plt.imshow(np.invert(dn_im), 'jet', interpolation='none', alpha=0.7)
          else:
            plt.imshow(dn_im, 'jet', interpolation='none', alpha=0.7)
          bbox = dict(boxstyle ="round", fc ="0.8")
          plt.annotate('Confluency: {cfy: .3f}\nDate: {dt}'.format(cfy=cfy, dt=dt), (0,0), bbox=bbox)
          plt.axis('off')
          #plt.show()
          #print("Image path: " + image_paths)
          #print("Date-Time: {dt}".format(dt=dt))
          #print("Confluency: {cfy:.3f}".format(cfy=cfy))
          buf = io.BytesIO()
          plt.savefig(buf, format="jpg", bbox_inches='tight')
          verbose_ims.append(widgets.Image(
            value=IPython.display.Image(buf.getvalue()).data,
            format='jpg'
          ))
          plt.clf()

        #clear_output()

  tqdm.write('\n')

  display("Finished")
  data_list = relative_days(data_dict['+'.join(list(image_set))])
  return data_list

#### Plotting

In [ ]:
from numpy.lib.function_base import percentile
# Code to plot confluency vs time
# Data points are averaged over time points within a 0.1 day tolerance range
def plot_func(data_list):
  x_list = []
  y_list = []
  y_err = []

  x_all = []
  y_all = []

  temp_x_arr = []
  temp_y_arr = []
  prev_val = 0.0
  tol = 0.1
  perc_val = 25 # hardcoded plus-or-minus from 50th percentile value
  for coord in sorted(data_list, key=lambda x: x[1]):
    x_all.append(coord[1])
    y_all.append(coord[0])
    if np.isclose(coord[1], prev_val, atol=tol):
      temp_x_arr.append(coord[1])
      temp_y_arr.append(coord[0])
      prev_val = coord[1]
    else:
      if median_mode.value:
        xdata = np.median(temp_x_arr)
        ydata = np.median(temp_y_arr)
        #yedata = np.subtract(np.percentile(temp_y_arr, 50 + perc_val, interpolation='linear'), np.percentile(temp_y_arr, 50 - perc_val, interpolation='linear'))
        yedata = 1.253*np.std(temp_y_arr)/np.sqrt(len(temp_y_arr))
      else:
        xdata = np.average(temp_x_arr)
        ydata = np.average(temp_y_arr)
        yedata = np.std(temp_y_arr)/np.sqrt(len(temp_y_arr))
      x_list.append(xdata)
      y_list.append(ydata)
      y_err.append(yedata)
      temp_x_arr = [coord[1]]
      temp_y_arr = [coord[0]]
      prev_val = coord[1]

  if median_mode.value:
    xdata = np.median(temp_x_arr)
    ydata = np.median(temp_y_arr)
    #yedata = np.subtract(np.percentile(temp_y_arr, 50 + perc_val, interpolation='linear'), np.percentile(temp_y_arr, 50 - perc_val, interpolation='linear'))
    yedata = 1.253*np.std(temp_y_arr)/np.sqrt(len(temp_y_arr))
  else:
    xdata = np.average(temp_x_arr)
    ydata = np.average(temp_y_arr)
    yedata = np.std(temp_y_arr)/np.sqrt(len(temp_y_arr))

  x_list.append(xdata)
  y_list.append(ydata)
  y_err.append(yedata)
      

  # calculate asymmetric error bars for data points in log2
  log2y_err = [np.log2(np.divide(y_list,np.subtract(y_list, y_err))), np.log2(np.divide(np.add(y_list, y_err),y_list))]

  if bin_by_date:
    plt.scatter(x_list, np.log2(y_list), color='orange')
    plt.errorbar(x_list, np.log2(y_list), yerr=log2y_err, fmt='o', color='orange')
  else:
    plt.scatter(x_all, np.log2(y_all), color='orange')

  # only if verbose mode is enabled
  if verbose_mode.value:
    # overlay individual data points on top of reduced plot
    plt.scatter(x_all, np.log2(y_all), marker='.' ,color='red', alpha=0.2)

  a, b = np.polyfit(x_list, np.log2(y_list), 1)

  x = np.linspace(0, max(x_list))
  y = a*x + b

  corr_matrix = np.corrcoef(y_list, [a*x_val + b for x_val in x_list])
  corr = corr_matrix[0,1]
  R_sq = corr**2

  if len(list(analyzer.kwargs.values())[0]) > 1:
    plt.title("Merged Analysis")
  else:
    plt.title(list(analyzer.kwargs.values())[0][0])

  plt.ylim(-7,0)
  plt.xlabel("Time (days)")
  plt.ylabel(r"Log$_2$ of confluency")
  plt.plot(x, y, label='y = {a:.3f} x + {b:.3f}\nR$^2$ = {c:.3f}'.format(a=a,b=b,c=R_sq))
  plt.legend(loc='lower right')
  plt.show()

  if verbose_mode.value:
    update_sheet(x_list, y_list)

In [ ]:
def data_analyze(image_set):
#  if not uploads.value:
#      print("No files uploaded")
#      return
#  if image_set == "Uploaded Image(s)":
#    print(uploads.value.items())
#    #im_dict["Uploaded Image(s)"] = list(uploads.value)
  display("Starting analysis...")
  data_list = im_set_analyze(image_set)
  plot_func(data_list)

  plot_grid = widgets.GridBox(verbose_ims, layout=widgets.Layout(grid_template_columns="repeat(4, 350px)"))
  verbose_outs = widgets.Accordion(children=[plot_grid])
  verbose_outs.set_title(0, 'Detailed outputs')
  display(verbose_outs)

###Google Sheets Interfacing

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
def update_sheet(x_all, y_all):
  #gc.create('confluency_data/' + list(analyzer.kwargs.values())[0][0]).sheet1
  #worksheet = gc.open('confluency_data/' + list(analyzer.kwargs.values())[0][0]).sheet1
  worksheet = gc.open('confluency_data').sheet1
  cell_titles = [["Time (days)", "Confluency"]]
  if worksheet.get_all_values():
    print('Google sheet already exists in directory')
  else:
    values = np.array([x_all, y_all]).T.tolist()
    worksheet.append_rows(cell_titles)
    worksheet.append_rows(values)

# Instructions
First, run the entire note book by selecting "*Runtime*" then "*Run all*" in the toolbar at the top of the notebook. Then, select the image set you wish to analyze. Once you made your selection, click "*Run Interact*".

**CURRENTLY ONLY SUPPORTS IMAGE SETS IN THE CELL TEAM DIRECTORY**

In order to output data points to google sheets, enable "detailed outputs" mode.

This will take about 5-20 minutes to run, so grab some coffee and go watch a show in the meantime.

*(Currently there is no "Abort" button, so once you start it, you will have to click "Restart and run all" in the toolbar if you want cancel the analysis).*

In [ ]:
#@title **Analyzer**
from ipywidgets import HBox, Label, IntProgress

#uploads = widgets.FileUpload(
#    accept='.jpg',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
#    multiple=True  # True to accept multiple files upload else False
#)

# analyzer interact
#analyzer = interactive(data_analyze, {'manual': True}, image_set=image_sets)
analyzer = interactive(data_analyze, {'manual': True}, image_set=widgets.SelectMultiple(value=[], options=image_sets, layout = widgets.Layout(height='300px', width='1000px'), description='Image Sets'))

# verbose checkbox
verbose_mode = widgets.Checkbox(
    value=False,
    description='Enable detailed outputs',
    disabled=False,
    indent=False)

# merge analysis checkbox
bin_by_date = widgets.Checkbox(
    value=True,
    description='Bin by date',
    disabled=False,
    indent=False)

# model dropdown
choose_model = widgets.Dropdown(
    options=['HEPG2', 'DAPI'],
    value='HEPG2',
    description='Model',
    disabled=False,
)

# enable median use instead of mean checkbox
median_mode = widgets.Checkbox(
    value=False,
    description='Use median',
    disabled=False,
    indent=False)

output = analyzer.children[-1]
output.layout.height = '500px'
#display(widgets.HBox([widgets.VBox([verbose_mode, median_mode, bin_by_date]), choose_model]), analyzer)
display(widgets.HBox([widgets.VBox([verbose_mode, median_mode, bin_by_date])]), analyzer)

interactive(children=(SelectMultiple(description='Image Sets', layout=Layout(height='300px', width='1000px'), …

In [ ]:
np.version.version

'1.21.6'